In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import json_normalize
import ast

In [2]:
#pd.reset_option('^display.', silent=True)
credits_df = pd.read_csv("credits.csv")
credits_df = pd.DataFrame(credits_df)
credits_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [3]:
# Dropping Duplicates
credits_df = credits_df.drop_duplicates(keep="first")

In [4]:
# Dropping Duplicates
credits_df = credits_df.drop_duplicates(keep="first")

In [5]:
credits_df["id"].nunique()

45432

Unnesting Cast

In [6]:
df_cast = credits_df.loc[:,["cast","id"]]
df_cast = df_cast.rename(columns={"id":"Movie_id"})

In [7]:
df_cast.head()

,cast,Movie_id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...",11862


In [8]:
# exploring shape
df_cast.shape

(45439, 2)

In [9]:
# Transform NaN values into empty lists
df_cast = df_cast.fillna('[]')

In [10]:
df_cast.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45439 entries, 0 to 45475
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cast      45439 non-null  object
 1   Movie_id  45439 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [11]:
# Transforming str into dictionaries

df_cast['cast'] = df_cast['cast'].apply(ast.literal_eval)

In [12]:
# Explode/Unnest cast

df_exploded1 = df_cast.explode('cast')

def extract_cast_info(x):
    if isinstance(x, dict):
        return pd.Series([x.get('cast_id', ''), x.get('character', ''), x.get('credit_id', ''), x.get('gender', ''), x.get('id', ''), x.get('name', ''), x.get('order', ''), x.get('profile_path', '')])
    else:
        return pd.Series(['', '', '', '', '', '', '', ''])

df_exploded1[['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order', 'profile_path']] = df_exploded1['cast'].apply(extract_cast_info)


In [13]:
df_exploded1.drop(["cast"],axis=1,inplace=True)
df_cast = df_exploded1
df_cast.head()

,Movie_id,cast_id,character,credit_id,gender,id,name,order,profile_path
0,862,14,Woody (voice),52fe4284c3a36847f8024f95,2,31,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
0,862,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2,12898,Tim Allen,1,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
0,862,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2,7167,Don Rickles,2,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
0,862,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2,12899,Jim Varney,3,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
0,862,18,Rex (voice),52fe4284c3a36847f8024fa5,2,12900,Wallace Shawn,4,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg


In [14]:
df_cast

,Movie_id,cast_id,character,credit_id,gender,id,name,order,profile_path
0,862,14,Woody (voice),52fe4284c3a36847f8024f95,2,31,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
0,862,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2,12898,Tim Allen,1,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
0,862,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2,7167,Don Rickles,2,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
0,862,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2,12899,Jim Varney,3,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
0,862,18,Rex (voice),52fe4284c3a36847f8024fa5,2,12900,Wallace Shawn,4,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
...,...,...,...,...,...,...,...,...,...
45474,227506,3,,52fe4ea59251416c7515d7d9,1,1090923,Nathalie Lissenko,1,None
45474,227506,4,,52fe4ea59251416c7515d7dd,2,1136422,Pavel Pavlov,2,None
45474,227506,5,,52fe4ea59251416c7515d7e1,0,1261758,Aleksandr Chabrov,3,None
45474,227506,6,,52fe4ea59251416c7515d7e5,1,29199,Vera Orlova,4,/n1NXVGNzNxtqsMWxLT1h8GO8Kpi.jpg


In [16]:
# DF Cast to csv
# This DF exceeded the github size, so i filtered it dow to Movie_id, character, name

In [17]:
df_cast = df_cast.loc[:,["Movie_id","character","name"]]

In [19]:
df_cast.to_csv("ETL_credits_cast.csv",index=False)

Unnesting Crew

In [20]:
df_crew = credits_df.loc[:,["crew","id"]]
df_crew = df_crew.rename(columns={"id":"Movie_id"})

In [21]:
df_crew.head()

,crew,Movie_id
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [22]:
df_crew.shape

(45439, 2)

In [23]:
df_crew.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45439 entries, 0 to 45475
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   crew      45439 non-null  object
 1   Movie_id  45439 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [24]:
# Transform NaN values into empty lists
df_crew = df_crew.fillna('[]')

In [25]:
# Transforming str into dictionaries

df_crew['crew'] = df_crew['crew'].apply(ast.literal_eval)

In [26]:
# Explode/Unnest cast
df_exploded2 = df_crew.explode('crew')

def extract_country_info(x):
    if isinstance(x, dict):
        return pd.Series([x.get('credit_id', ''), x.get('department', ''),x.get('gender', ''),x.get('id', ''),x.get('job', ''),x.get('name', ''),x.get('profile_path', '')  ])
    else:
        return pd.Series(['', ''])

df_exploded2[['credit_id', 'department', 'gender','id','job', 'name', 'profile_path']] = df_exploded2['crew'].apply(extract_country_info)


In [27]:
df_exploded2.head()

,crew,Movie_id,credit_id,department,gender,id,job,name,profile_path
0,"{'credit_id': '52fe4284c3a36847f8024f49', 'dep...",862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
0,"{'credit_id': '52fe4284c3a36847f8024f4f', 'dep...",862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
0,"{'credit_id': '52fe4284c3a36847f8024f55', 'dep...",862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
0,"{'credit_id': '52fe4284c3a36847f8024f5b', 'dep...",862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
0,"{'credit_id': '52fe4284c3a36847f8024f61', 'dep...",862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


In [28]:
df_exploded2.drop(["crew"],axis=1,inplace=True)
df_exploded2

,Movie_id,credit_id,department,gender,id,job,name,profile_path
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
0,862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
0,862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
0,862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
0,862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg
...,...,...,...,...,...,...,...,...
45473,67758,52fe4776c3a368484e0c8399,Sound,0.0,549356.0,Original Music Composer,Richard McHugh,None
45473,67758,52fe4776c3a368484e0c839f,Camera,2.0,58818.0,Director of Photography,João Fernandes,None
45474,227506,533bccebc3a36844cf0011a7,Directing,0.0,1085341.0,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg
45474,227506,58ebbc26925141281908aa0a,Production,2.0,1195656.0,Producer,Joseph N. Ermolieff,None


In [29]:
df_crew = df_exploded2
df_crew.head()

,Movie_id,credit_id,department,gender,id,job,name,profile_path
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
0,862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
0,862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
0,862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
0,862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


In [28]:
# DF Crew to csv
# This DF exceeded the github size, so i filtered it dow to Movie_id, department, job, name

In [31]:
df_crew = df_crew.loc[:,["Movie_id","department","job","name"]]

,Movie_id,department,job,name
0,862,Directing,Director,John Lasseter
0,862,Writing,Screenplay,Joss Whedon
0,862,Writing,Screenplay,Andrew Stanton
0,862,Writing,Screenplay,Joel Cohen
0,862,Writing,Screenplay,Alec Sokolow
...,...,...,...,...
45473,67758,Sound,Original Music Composer,Richard McHugh
45473,67758,Camera,Director of Photography,João Fernandes
45474,227506,Directing,Director,Yakov Protazanov
45474,227506,Production,Producer,Joseph N. Ermolieff


In [32]:
df_crew.to_csv("ETL_credits_crew.csv",index=False)